In [1]:
import torch
import torchvision.transforms as T
from PIL import Image
import requests
from pycocotools.coco import COCO
import os
import time
import sys

# DETR 모델 로드
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

# 이미지 변환 함수 (DETR에 맞게 변환)
transform = T.Compose([
    T.Resize(800),  # DETR 모델은 800px 크기로 입력
    T.ToTensor(),
])

# COCO 데이터셋 로드
data_dir = "coco"
ann_file = "coco/annotations/instances_val2017.json"
coco = COCO(ann_file)

# 첫 번째 이미지 선택
img_ids = coco.getImgIds()
img_info = coco.loadImgs(img_ids[0])[0]
img_path = os.path.join(data_dir, "val2017", img_info["file_name"])

# 이미지 로드 및 변환
image = Image.open(img_path).convert("RGB")
img_tensor = transform(image).unsqueeze(0).to(device)  # 배치 차원 추가

# 모델 추론
with torch.no_grad():
    start_event.record()
    outputs = model(img_tensor)
    end_event.record()

# 출력 확인
# print("Model Outputs:", outputs)
time_taken = start_event.elapsed_time(end_event)
print(f'Elapsed time on GPU: {time_taken} ms')

Using cache found in C:\Users\avdc/.cache\torch\hub\facebookresearch_detr_main
C:\Users\avdc\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\avdc\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
Elapsed time on GPU: 182.82191467285156 ms
